In [5]:
%matplotlib inline

In [6]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.linear_model
from datetime import datetime
from sklearn_pandas import DataFrameMapper
import warnings
import timeit
from collections import defaultdict
import tabulate

timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

matplotlib.style.use('ggplot')

RANDOM_SEED = 33

In [35]:
from numpy.random import negative_binomial, choice

IN = 'in'
OUT = 'out'
ALPHA = 'alpha'
BETA = 'beta'
N = 'north'
S = 'south'
E = 'east'
W = 'west'
DIRECTIONS = (IN, OUT)
WIND_DIRECTIONS = (N, S, E, W)

POSTERIOR_PARAMETERS = {
    IN: {
        N: {ALPHA: 73, BETA: 13},
        S: {ALPHA: 76, BETA: 13},
        E: {ALPHA: 81, BETA: 13},
        W: {ALPHA: 88, BETA: 13},
    },
    OUT: {
        N: {ALPHA: 88, BETA: 13},
        S: {ALPHA: 83, BETA: 13},
        E: {ALPHA: 65, BETA: 13},
        W: {ALPHA: 65, BETA: 13},
    },
}

IN_TRAFFIC_DISTRIBUTIONS = {
    N: {
        S: 0.5,
        E: 0.25,
        W: 0.25,
    },
    S: {
        N: 0.5,
        E: 0.25,
        W: 0.25,
    },
    E: {
        W: 0.5,
        N: 0.25,
        S: 0.25,
    },
    W: {
        E: 0.5,
        N: 0.25,
        S: 0.25,
    },
}


def posterior_predictive_sample(direction, wind_direction, size=None):
    if direction not in (IN, OUT) or wind_direction not in (N, S, E, W):
        raise ValueError('Invalid direction ({d}) or wind direction ({wd})'
                            .format(d=direction, wd=wind_direction))
        
    alpha = POSTERIOR_PARAMETERS[direction][wind_direction][ALPHA]
    beta = POSTERIOR_PARAMETERS[direction][wind_direction][BETA]
    return negative_binomial(alpha, beta / (1.0 + beta))


def in_to_out_traffic_sample(in_wind_direction, incoming_car_count=None):
    if incoming_car_count is None:
        incoming_car_count = posterior_predictive_sample(IN, in_wind_direction)
        
    out_directions = IN_TRAFFIC_DISTRIBUTIONS[in_wind_direction].keys()
    directions, probs = zip(*IN_TRAFFIC_DISTRIBUTIONS[in_wind_direction].items())
    return choice(directions, size=incoming_car_count, p=probs)


In [38]:
VERBOSE = False

TIME_STEPS = 20
CONGESTION_THRESHOLD = 8
BLOCKED_THRESHOLD = 15

traffic_count = defaultdict(lambda: defaultdict(lambda: 0))
congestion_counts = defaultdict(lambda: defaultdict(lambda: 0))
blocked_counts = defaultdict(lambda: defaultdict(lambda: 0))


def resolve_cars_from_direction(in_wd, cars_from_direction, t):
    for index, out_wd in enumerate(cars_from_direction):
        # if it goes over the blocked threshold, stop incoming traffic (as it's waiting for this car to turn)
        if traffic_count[OUT][out_wd] >= BLOCKED_THRESHOLD:
            if VERBOSE: print('At t={t}, {out_wd}_out is blocked'.format(t=t, out_wd=out_wd))
            blocked_counts[OUT][out_wd] += 1
            # returning the number of cars that were succesfully resoved
            return index
        
        # if it goes over the congested threshold, report congestion / slowing down
        if traffic_count[OUT][out_wd] >= CONGESTION_THRESHOLD:
            if VERBOSE: print('At t={t}, {out_wd}_out is congested'.format(t=t, out_wd=out_wd))
            congestion_counts[OUT][out_wd] += 1
            
        traffic_count[OUT][out_wd] += 1
        
    return len(cars_from_direction)


# main body of simulation
for t in range(TIME_STEPS):
    # at every time step, simulate number of cars that can leave in each direction 
    # (allowing count to be negative if need be)
    for out_wd in WIND_DIRECTIONS:
        traffic_count[OUT][out_wd] -= posterior_predictive_sample(OUT, out_wd)
        
    for in_wd in WIND_DIRECTIONS:
        # first, try to clear up existing backed up traffic
        if traffic_count[IN][in_wd] > 0:
            cars_from_direction = in_to_out_traffic_sample(in_wd, traffic_count[IN][in_wd])
            traffic_count[IN][in_wd] -= resolve_cars_from_direction(in_wd, cars_from_direction, t)
            
        # if all backed up traffic cleared, simulate new incoming traffic leaving:
        if traffic_count[IN][in_wd] == 0:
            cars_from_direction = in_to_out_traffic_sample(in_wd)
            resolved_count = resolve_cars_from_direction(in_wd, cars_from_direction, t)
            traffic_count[IN][in_wd] += len(cars_from_direction) - resolved_count
                
        # otherwise, just generate incoming traffic and back it up
        else:
             traffic_count[IN][in_wd] += posterior_predictive_sample(IN, in_wd)
                
        # increment blocked or congested count
        final_in_count = traffic_count[IN][in_wd]
        if final_in_count >= BLOCKED_THRESHOLD:
            if VERBOSE: print('At t={t}, {in_wd}_in is blocked'.format(t=t, in_wd=in_wd))
            blocked_counts[IN][in_wd] += 1
        elif final_in_count >= CONGESTION_THRESHOLD:
            if VERBOSE: print('At t={t}, {in_wd}_in is congested'.format(t=t, in_wd=in_wd))
            congestion_counts[IN][in_wd] += 1
            
    # at this point, if any out direction is still negative 
    # (had fewer cars entering than could leave) - zero it
    for out_wd in WIND_DIRECTIONS:
        traffic_count[OUT][out_wd] = max(traffic_count[OUT][out_wd], 0)


# report 
print('After t = {t} time steps, we observed the following data:'.format(t=TIME_STEPS))
headers = ('Direction', 'Current traffic', 'Total blocked count', 'Total congested count')
table = []
for wd in WIND_DIRECTIONS:
    for d in DIRECTIONS:
        table.append(['{wd}_{d}'.format(wd=wd, d=d), traffic_count[d][wd], 
                      blocked_counts[d][wd], congestion_counts[d][wd]])
        
print(tabulate.tabulate(table, headers, tablefmt='fancy_grid'))




After t = 20 time steps, we observed the following data:
╒═════════════╤═══════════════════╤═══════════════════════╤═════════════════════════╕
│ Direction   │   Current traffic │   Total blocked count │   Total congested count │
╞═════════════╪═══════════════════╪═══════════════════════╪═════════════════════════╡
│ north_in    │                 0 │                     0 │                       0 │
├─────────────┼───────────────────┼───────────────────────┼─────────────────────────┤
│ north_out   │                11 │                     0 │                       3 │
├─────────────┼───────────────────┼───────────────────────┼─────────────────────────┤
│ south_in    │                 0 │                     0 │                       1 │
├─────────────┼───────────────────┼───────────────────────┼─────────────────────────┤
│ south_out   │                 0 │                     0 │                       0 │
├─────────────┼───────────────────┼───────────────────────┼────────────────────────